In [75]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from scipy import stats
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.multiclass import OneVsOneClassifier

import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay

In [76]:
TEST_SIZE = 0.2

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def vectorization_TF_IDF(DATASET, text_field_name, label_name):

    X_train, X_test, y_train, y_test = train_test_split( DATASET[text_field_name] ,
                                                    DATASET[label_name],
                                                    train_size=0.80, random_state=28)

    encoder = preprocessing.LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)

    return X_train, X_test, y_train, y_test

def saving_results(results, path, file_name):

    results.to_csv(path + file_name,  index = False, sep=";")

def train_model(classifier, X_train_v, X_test_v, y_train, y_test):
    ini = time.time()
    classifier.fit(X_train_v, y_train)
    predictions = classifier.predict(X_test_v)
    fim = time.time()

    timee = fim-ini
    acuracia = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, predictions)
    recall = recall_score(y_test, predictions, average='weighted')
    f1_score = metrics.f1_score(y_test, predictions, average='weighted')
    # print("Time: " + str(timee))
    # print("Acurácia: " + str(acuracia))
    # print("Precision: " + str(precision))
    # print("Recal: " + str(recall))
    # print("f1_score: " + str(f1_score))

    return timee , acuracia, precision, recall, f1_score, matriz_confusao

def get_tests_result(X_train_v, X_test_v, y_train, y_test):

    all_res = []
    ### MLPClassifier

    #print( "MLPClassifier")
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(70, ), random_state=1, verbose=True)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    all_res.append(["MLPClassifier: ", timee , acuracia, precision, recall, f1_score])



    ### AdaBoostClassifier

    #print( "AdaBoostClassifier: ")
    clf = AdaBoostClassifier(n_estimators= 50, learning_rate=1)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    all_res.append(["AdaBoostClassifier: ", timee , acuracia, precision, recall, f1_score])



    ### Voting_LR3_SVC1_ETC2
    #print( "Voting_LR3_SVC1_ETC2: ")
    ini = time.time()
    clf2 = LogisticRegression(random_state=0)
    clf5 = SVC(kernel='rbf', probability=True)
    clf6 = ExtraTreesClassifier(n_estimators=100, random_state=0)
    eclf = VotingClassifier(estimators=[ ('lr', clf2), ('svc', clf5), ('etc', clf6)], voting='soft', weights=[3, 1, 2])
    clf2 = clf2.fit(X_train_v, y_train)
    clf5 = clf5.fit(X_train_v, y_train)
    clf6 = clf6.fit(X_train_v, y_train)
    eclf = eclf.fit(X_train_v, y_train)
    #Y_previsto_vc1 = eclf.predict(X_test_v.toarray())
    Y_previsto_vc1 = eclf.predict(X_test_v)
    fim = time.time()

    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_previsto_vc1)
    precision = precision_score(y_test, Y_previsto_vc1, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_previsto_vc1)
    recall = recall_score(y_test, Y_previsto_vc1, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_previsto_vc1, average='weighted')
    # print("Time: " + str(timee))
    # print("Acurácia: " + str(acuracia))
    # print("Precision: " + str(precision))
    # print("Recal: " + str(recall))
    # print("f1_score: " + str(f1_score))
    # print(matriz_confusao)
    all_res.append(["Voting_LR3_SVC1_ETC2: ", timee , acuracia, precision, recall, f1_score])



    ### DecisionTreeClassifier
    #print( "DecisionTreeClassifier: ")
    clf = DecisionTreeClassifier(random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    all_res.append(["DecisionTreeClassifier: ", timee , acuracia, precision, recall, f1_score])



    ### RandomForestClassifier
    #print( "RandomForestClassifier: " )
    clf = RandomForestClassifier(n_estimators=50, random_state=1)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    all_res.append(["RandomForestClassifier: ", timee , acuracia, precision, recall, f1_score])



    ### ExtraTreesClassifier
    #print( "ExtraTreesClassifier: ")
    clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    all_res.append(["ExtraTreesClassifier: ", timee , acuracia, precision, recall, f1_score])



    ### LogisticRegression
    #print( "LogisticRegression: ")
    clf = LogisticRegression(random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    all_res.append(["LogisticRegression: ",  timee , acuracia, precision, recall, f1_score])



    ### svm
    #print( "svm: ")
    clf = svm.SVC()
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    all_res.append(["svm: ",  timee , acuracia, precision, recall, f1_score])



    ### Stacking_scikit
    #print( "Stacking_scikit: ")
    clf1 = RandomForestClassifier(n_estimators=50, random_state=1)
    clf2 = LogisticRegression(random_state=0)
    clf3 = SVC()
    clf4 = ExtraTreesClassifier(n_estimators=100, random_state=0)
    estimators = [('rf', clf1), ('lr', clf2), ('svc', clf3), ('etc', clf4)]
    ini = time.time()
    clf = StackingClassifier( estimators=estimators, final_estimator=LogisticRegression(), n_jobs=None)
    clf.fit(X_train_v, y_train)
    Y_Previsto_stacking = clf.predict(X_test_v)
    fim = time.time()

    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_Previsto_stacking)
    precision = precision_score(y_test, Y_Previsto_stacking, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_Previsto_stacking)
    recall = recall_score(y_test, Y_Previsto_stacking, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_Previsto_stacking, average='weighted')
    # print("Time: " + str(timee))
    # print("Acurácia: " + str(acuracia))
    # print("Precision: " + str(precision))
    # print("Recal: " + str(recall))
    # print("f1_score: " + str(f1_score))
    # print(matriz_confusao)
    all_res.append(["Stacking_scikit: ",  timee , acuracia, precision, recall, f1_score])



    ### XGBClassifier
    #print( "XGBClassifier: ")
    #clf = XGBClassifier(eval_metric='mlogloss' )
    #timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    #all_res.append(["XGBClassifier: ",  timee , acuracia, precision, recall, f1_score])



    ### OneVsRestClassifier_RF
    #print( "OneVsRest_RF: ")
    clf = RandomForestClassifier(n_estimators=50, random_state=1)
    ovr = OneVsRestClassifier(clf)
    ini = time.time()
    ovr.fit(X_train_v, y_train)
    Y_Previsto_OVR_RF = ovr.predict(X_test_v)
    fim = time.time()


    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_Previsto_OVR_RF)
    precision = precision_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_Previsto_OVR_RF)
    recall = recall_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    # print("Time: " + str(timee))
    # print("Acurácia: " + str(acuracia))
    # print("Precision: " + str(precision))
    # print("Recal: " + str(recall))
    # print("f1_score: " + str(f1_score))
    # print(matriz_confusao)
    all_res.append(["OvR_RF: ",  timee , acuracia, precision, recall, f1_score])

    return pd.DataFrame( all_res, columns=["ALGORITHM","TIME","ACCURACY","PRECISION","RECALL", "F1_SCORE"])

feat = ['Idade', 'Peso Kg', 'Cor da pele', 'ACTN3',
       'AGT', 'ECA', 'Score Força%', 'Score Resistência %',
       'Pré CK (U/L)', 'Pós CK (U/L)', 'Pré CK MB U/L', 'Pós CK MB U/L',
       'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', 'Pré LDH (U/L)',
       'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L',
       'Osmolalidade sérica PRÉ BD4(mOsm/kg): 1,86(Na + K) + 1,15(Gli /18) + (Ureia/6) + 14\t\t\t',
       'Osmolalidade sérica PÓS BD4(mOsm/kg): 1,86(Na + K) + 1,15(Gli /18) + (Ureia/6) + 14\t\t\t',
       'Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %',
       'Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %', 'massa magra', 'MCT KG', 'ESTATURA', 'vo2max estimado',
       'Handgrip     Dom.', 'Handgrip Cont.', '% gordura']

label_rabdomiolise = "Rabdomiólise"
label_lesao = "Lesão Renal Aguda"




In [77]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/'
file = 'dados.xlsx'
file2 = 'dados_completos.xlsx'
DATASET = pd.read_excel(path + file)
DATASET.drop(columns=['Ano','Missão','Aluno'])

DATASET2 = pd.read_excel(path + file2)
DATASET


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ano  Missão  Aluno Idade Peso Kg Cor da pele ACTN3  AGT  ECA   BDKRB2  \
0    2018   CIASC   3150    33      88           1    XX   MT   ID  (-9,-9)   
1    2018   CIASC   3155    26      74           1    RR   MT   II  (+9,-9)   
2    2018   CIASC   3156    26      76           1    RX   MM   ID  (+9,+9)   
3    2018   CIASC   3159    25      84           1    RR   TT   DD  (+9,-9)   
4    2018   CIASC   3165    33      82           1    RR   MT   II  (+9,-9)   
..    ...     ...    ...   ...     ...         ...   ...  ...  ...      ...   
462  2023  Manaus   3340    29     NaN           2   NaN  NaN  NaN      NaN   
463  2023  Manaus   3343    27     NaN           2   NaN  NaN  NaN      NaN   
464  2023  Manaus   3347    28     NaN           2   NaN  NaN  NaN      NaN   
465  2023  Manaus   3348    29     NaN           2   NaN  NaN  NaN      NaN   
466  2023  Manaus   3350    29     NaN           2   NaN  NaN  NaN      NaN   

     ... Pós BASO % Lesão Renal Aguda Rabdomiólise  massa magra MCT KG  \
0    ...        0.1             False        False        75.52   84.3   
1    ...        0.2             False         True        69.49   75.9   
2    ...        0.3             False         True        72.07   79.2   
3    ...        0.9             False         True        75.26   84.3   
4    ...        0.1             False         True        66.26   79.6   
..   ...        ...               ...          ...          ...    ...   
462  ...        NaN             False        False        76,40   82.8   
463  ...        NaN             False        False        65,80   75.1   
464  ...        NaN             False        False        74,27   82.7   
465  ...        NaN             False        False        70,18   80.5   
466  ...        NaN             False        False        75,16   85.8   

     ESTATURA  vo2max estimado  Handgrip     Dom. Handgrip Cont.  % gordura  
0       181.5            50.85               35.0           37.0      10.05  
1       171.0            51.98               42.0           43.0       6.08  
2       174.0            52.42               49.5           49.5       8.72  
3       178.0            50.68               60.0           57.0      10.61  
4       182.0            47.64               51.0           49.0      14.84  
..        ...              ...                ...            ...        ...  
462     182.0            53,54               56.0           59.0       7,73  
463     170.0                -               48.0           60.0      12,39  
464     179.5            50,56               58.0           52.0      10,20  
465     181.0            51,80               60.0           50.0      12,82  
466     178.0            53,41               55.0           56.0      12,40  

[467 rows x 96 columns]

**PREPRAÇÃO DOS DADOS**

CASO 1 - Preencher NaN

In [78]:
from sklearn.impute import SimpleImputer


DATASET_fill_mean = DATASET.copy()
DATASET_fill_mean = DATASET_fill_mean.replace('-', np.nan)

# Preencher numéricos com média
imputer_mean = SimpleImputer(strategy='mean')
X_numeric = DATASET_fill_mean.select_dtypes(include=['float64', 'int64'])
DATASET_fill_mean[X_numeric.columns] = imputer_mean.fit_transform(X_numeric)

# Preencher categóricos com moda
imputer_freq = SimpleImputer(strategy='most_frequent')
X_categorical = DATASET_fill_mean.select_dtypes(include=['object', 'category'])
DATASET_fill_mean[X_categorical.columns] = imputer_freq.fit_transform(X_categorical)

X_fill_mean = DATASET_fill_mean.drop(columns=['Rabdomiólise'])
y_fill_mean = DATASET_fill_mean['Rabdomiólise']

X_fm_Train, X_fm_Test, y_fm_Train, y_fm_Test = train_test_split(X_fill_mean,y_fill_mean,test_size=TEST_SIZE)

CASO 2 - Dropar colunas com 50% mais de NaN e depois dropar linhas com NaN

In [79]:
taxa_drop = 0.5

DATASET_drop_cols50 = DATASET.copy()

DATASET_drop_cols50 = DATASET_drop_cols50.loc[:, DATASET_drop_cols50.isnull().mean() < taxa_drop]

DATASET_drop_cols50 = DATASET_drop_cols50.dropna()

#Codificar colunas categóricas
label_encoder = preprocessing.LabelEncoder()
for coluna in DATASET_drop_cols50.select_dtypes(include=['object', 'category']).columns:
    DATASET_drop_cols50[coluna] = label_encoder.fit_transform(DATASET_drop_cols50[coluna].astype(str))

X_drop_cols50 = DATASET_drop_cols50.drop(columns=['Rabdomiólise'])
y_drop_cols50 = DATASET_drop_cols50['Rabdomiólise']

X_drop_cols50_Train, X_drop_cols50_Test, y_drop_cols50_Train, y_drop_cols50_Test = train_test_split(X_drop_cols50,y_drop_cols50,test_size=TEST_SIZE)

X_drop_cols50_Train.shape

(204, 95)

CASO 3 - fill_mean + transforma texto/categotias em números

In [80]:
DATASET_encoded = DATASET_fill_mean.copy()

# Codificar colunas categóricas
label_encoder = preprocessing.LabelEncoder()
for coluna in DATASET_encoded.select_dtypes(include=['object', 'category']).columns:
    DATASET_encoded[coluna] = label_encoder.fit_transform(DATASET_encoded[coluna].astype(str))

X_encoded = DATASET_encoded.drop(columns=['Rabdomiólise'])
scale_obj = preprocessing.StandardScaler()
X_encoded = scale_obj.fit_transform(X_encoded)

y_encoded = DATASET_encoded['Rabdomiólise']

X_encoded_Train, X_encoded_Test, y_encoded_Train, y_encoded_Test = train_test_split(X_encoded,y_encoded,test_size=TEST_SIZE)

X_encoded_Train.shape

(373, 95)

**APLICANDO PCA NOS DADOS**

In [81]:
NUMBER_COMPONENTS = len(X_encoded_Test)


import copy
# Guarda cópias dos dados originais
X_drop_cols50_Train_original = copy.deepcopy(X_drop_cols50_Train)
X_drop_cols50_Test_original = copy.deepcopy(X_drop_cols50_Test)
X_encoded_Train_original = copy.deepcopy(X_encoded_Train)
X_encoded_Test_original = copy.deepcopy(X_encoded_Test)

# Verifica e imprime os shapes originais
print("Shapes originais:")
print(f"X_drop_cols50_Train: {X_drop_cols50_Train_original.shape}")
print(f"X_encoded_Train: {X_encoded_Train_original.shape}")

# Loop para diferentes números de componentes
for i in range(1, NUMBER_COMPONENTS):
    print(f"\n===== Processando com {i} componentes =====")

    # Sempre use os dados originais para cada iteração
    try:
        # Para X_drop_cols50
        current_min = min(X_drop_cols50_Train_original.shape[0], X_drop_cols50_Train_original.shape[1])
        print(f"X_drop_cols50_Train shape atual: {X_drop_cols50_Train_original.shape}, min(n_samples, n_features) = {current_min}")

        if i <= current_min:
            # Usa os dados originais, não os transformados anteriormente
            pca_drop = decomposition.PCA(n_components=i)
            X_drop_cols50_Train_pca = pca_drop.fit_transform(X_drop_cols50_Train_original)
            X_drop_cols50_Test_pca = pca_drop.transform(X_drop_cols50_Test_original)

            print(f"PCA aplicado com sucesso em X_drop_cols50: resultado shape = {X_drop_cols50_Train_pca.shape}")

            # Avalia e salva resultados
            results_rabdo_dropcols = get_tests_result(X_drop_cols50_Train_pca, X_drop_cols50_Test_pca,
                                                    y_drop_cols50_Train, y_drop_cols50_Test)
            saving_results(results_rabdo_dropcols,
                          path + f"ResultadosTestesPIBITI/TesteComPCA/NUMBER_COMPONENTES_{i}_",
                          "resultados_rabdomiolise_dropcols.csv")
        else:
            print(f"Pulando PCA para X_drop_cols50: {i} componentes > {current_min}")

        # Para X_encoded
        current_min = min(X_encoded_Train_original.shape[0], X_encoded_Train_original.shape[1])
        print(f"X_encoded_Train shape atual: {X_encoded_Train_original.shape}, min(n_samples, n_features) = {current_min}")

        if i <= current_min:
            # Usa os dados originais, não os transformados anteriormente
            pca_encoded = decomposition.PCA(n_components=i)
            X_encoded_Train_pca = pca_encoded.fit_transform(X_encoded_Train_original)
            X_encoded_Test_pca = pca_encoded.transform(X_encoded_Test_original)

            print(f"PCA aplicado com sucesso em X_encoded: resultado shape = {X_encoded_Train_pca.shape}")

            # Avalia e salva resultados
            results_rabdo_encoded = get_tests_result(X_encoded_Train_pca, X_encoded_Test_pca,
                                                   y_encoded_Train, y_encoded_Test)
            saving_results(results_rabdo_encoded,
                          path + f"ResultadosTestesPIBITI/TesteComPCA/NUMBER_COMPONENTES_{i}_",
                          "resultados_rabdomiolise_encoded.csv")
        else:
            print(f"Pulando PCA para X_encoded: {i} componentes > {current_min}")

    except Exception as e:
        print(f"Erro na iteração {i}: {e}")

Shapes originais:
X_drop_cols50_Train: (204, 95)
X_encoded_Train: (373, 95)

===== Processando com 1 componentes =====
X_drop_cols50_Train shape atual: (204, 95), min(n_samples, n_features) = 95
PCA aplicado com sucesso em X_drop_cols50: resultado shape = (204, 1)
X_encoded_Train shape atual: (373, 95), min(n_samples, n_features) = 95
PCA aplicado com sucesso em X_encoded: resultado shape = (373, 1)

===== Processando com 2 componentes =====
X_drop_cols50_Train shape atual: (204, 95), min(n_samples, n_features) = 95
PCA aplicado com sucesso em X_drop_cols50: resultado shape = (204, 2)
X_encoded_Train shape atual: (373, 95), min(n_samples, n_features) = 95
PCA aplicado com sucesso em X_encoded: resultado shape = (373, 2)

===== Processando com 3 componentes =====
X_drop_cols50_Train shape atual: (204, 95), min(n_samples, n_features) = 95
PCA aplicado com sucesso em X_drop_cols50: resultado shape = (204, 3)
X_encoded_Train shape atual: (373, 95), min(n_samples, n_features) = 95
PCA aplic

### Teste 1 -  'Pré CK (U/L)', 'Pós CK (U/L)', "Rabdomiólise"

In [82]:
col_label_rabdo = ["Rabdomiólise"]
col_label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré CK (U/L)', 'Pós CK (U/L)']
col_to_be_encoded = ["Rabdomiólise", "Lesão Renal Aguda"]


col_df = ['Pré CK (U/L)', 'Pós CK (U/L)', "Rabdomiólise", "Lesão Renal Aguda"]


df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()

for c in col_df:
    df = df.drop(df[df[c] == '-'].index)


for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])


for d in col_feature:
    df[d] = df[d].astype(float)

df

Pré CK (U/L)  Pós CK (U/L)  Rabdomiólise  Lesão Renal Aguda
0           337.0         664.0             0                  0
1           687.0        2246.0             1                  0
2           841.0        4616.0             1                  0
3           363.0        2611.0             1                  0
4           192.0        1111.0             1                  0
..            ...           ...           ...                ...
423         223.0        1570.0             1                  0
424         340.0       11324.0             1                  1
425         312.0        4688.0             1                  0
426         109.0        6710.0             1                  0
427          90.0        4504.0             1                  0

[390 rows x 4 columns]

Prevendo Rabdomiólise

Prevendo Lesão Renal Aguda

Salvando os resultados

# Teste 2 - Pré CK MB e Pós CK MB

In [83]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré CK MB U/L', 'Pós CK MB U/L']
col_to_be_encoded = ["Rabdomiólise"]

col_df = ['Pré CK MB U/L', 'Pós CK MB U/L', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_feature:
    df[d] = df[d].astype(float)

df

Pré CK MB U/L  Pós CK MB U/L  Rabdomiólise
0              3.0           16.0             0
1              6.0           38.0             1
2             43.0           71.0             1
3              5.0           53.0             1
4              3.0           15.0             1
..             ...            ...           ...
339           20.0          153.0             1
340           20.0           45.0             0
341           18.0           57.0             0
342           19.0           76.0             1
343           38.0          135.0             1

[344 rows x 3 columns]

In [84]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.267747  0.594203   0.616937  0.594203  0.569521
1      AdaBoostClassifier:   0.184109  0.579710   0.610306  0.579710  0.543088
2    Voting_LR3_SVC1_ETC2:   0.598935  0.637681   0.639187  0.637681  0.637224
3  DecisionTreeClassifier:   0.009074  0.608696   0.631350  0.608696  0.587849
4  RandomForestClassifier:   0.151799  0.594203   0.616937  0.594203  0.569521
5    ExtraTreesClassifier:   0.218454  0.623188   0.666256  0.623188  0.593900
6      LogisticRegression:   0.009278  0.623188   0.623188  0.623188  0.623188
7                     svm:   0.010081  0.652174   0.674858  0.652174  0.642279
8         Stacking_scikit:   2.080753  0.623188   0.625501  0.623188  0.622237
9                  OvR_RF:   0.114321  0.594203   0.616937  0.594203  0.569521

# Teste 3 - 'ACTN3',  'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %,

In [85]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %']
col_to_be_encoded = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', "Rabdomiólise"]
col_to_float = ['Score Força%', 'Score Resistência %']

col_df = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

ACTN3  AGT  ECA  BDKRB2  Score Força%  Score Resistência %  Rabdomiólise
0        3    1    1       6          25.0                 75.0             0
1        0    1    2       3          50.0                 50.0             1
2        1    0    1       2          50.0                 50.0             1
4        0    1    2       3          50.0                 50.0             1
5        3    1    1       6         100.0                  0.0             1
..     ...  ...  ...     ...           ...                  ...           ...
338      3    2    0       3          62.5                 37.5             1
339      0    0    0       6          50.0                 50.0             1
340      3    2    1       6          37.5                 62.5             0
342      0    1    1       6          50.0                 50.0             1
343      3    2    0       3          62.5                 37.5             1

[268 rows x 7 columns]

In [86]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.217574  0.481481   0.460637  0.481481  0.465468
1      AdaBoostClassifier:   0.155159  0.500000   0.456349  0.500000  0.459100
2    Voting_LR3_SVC1_ETC2:   0.393711  0.444444   0.371153  0.444444  0.391450
3  DecisionTreeClassifier:   0.005089  0.444444   0.460077  0.444444  0.446734
4  RandomForestClassifier:   0.091183  0.425926   0.405350  0.425926  0.412342
5    ExtraTreesClassifier:   0.136197  0.444444   0.460077  0.444444  0.446734
6      LogisticRegression:   0.014734  0.500000   0.418478  0.500000  0.427734
7                     svm:   0.010831  0.574074   0.329561  0.574074  0.418736
8         Stacking_scikit:   1.573446  0.574074   0.329561  0.574074  0.418736
9                  OvR_RF:   0.114346  0.425926   0.405350  0.425926  0.412342

# Teste 4 - 'Idade', 'Peso Kg', 'Cor da pele',  'Score Força%', 'Score Resistência %'

In [87]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Idade', 'Peso Kg', 'Cor da pele', 'Score Força%', 'Score Resistência %']
col_to_be_encoded = ["Cor da pele", "Rabdomiólise"]
col_to_float = ['Idade', 'Peso Kg', 'Score Força%', 'Score Resistência %']

col_df = ['Idade', 'Peso Kg', 'Cor da pele', 'Score Força%', 'Score Resistência %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

Idade  Peso Kg  Cor da pele  Score Força%  Score Resistência %  \
0     33.0     88.0            1          25.0                 75.0   
1     26.0     74.0            1          50.0                 50.0   
2     26.0     76.0            1          50.0                 50.0   
4     33.0     82.0            1          50.0                 50.0   
5     31.0     92.0            1         100.0                  0.0   
..     ...      ...          ...           ...                  ...   
338   28.0     87.0            1          62.5                 37.5   
339   27.0     78.5            1          50.0                 50.0   
340   25.0     85.1            1          37.5                 62.5   
342   28.0     85.0            1          50.0                 50.0   
343   28.0     81.0            1          62.5                 37.5   

     Rabdomiólise  
0               0  
1               1  
2               1  
4               1  
5               1  
..            ...  
338             1  
339             1  
340             0  
342             1  
343             1  

[268 rows x 6 columns]

In [88]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.197232  0.500000   0.467857  0.500000  0.470879
1      AdaBoostClassifier:   0.135849  0.518519   0.485859  0.518519  0.485082
2    Voting_LR3_SVC1_ETC2:   0.422909  0.425926   0.405350  0.425926  0.412342
3  DecisionTreeClassifier:   0.004508  0.351852   0.367950  0.351852  0.349851
4  RandomForestClassifier:   0.090270  0.407407   0.413436  0.407407  0.409877
5    ExtraTreesClassifier:   0.134980  0.407407   0.418381  0.407407  0.410677
6      LogisticRegression:   0.012926  0.555556   0.512327  0.555556  0.482621
7                     svm:   0.010060  0.574074   0.329561  0.574074  0.418736
8         Stacking_scikit:   1.599161  0.574074   0.329561  0.574074  0.418736
9                  OvR_RF:   0.115105  0.407407   0.413436  0.407407  0.409877

# Teste 5 - 'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)'

In [89]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)']

col_df = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

Pré Lactato (mmol/L)  Pós Lactato (mmol/L)  Rabdomiólise
0                     0.0                   0.0             0
1                     0.0                   0.0             1
2                     0.0                   0.0             1
3                     0.0                   0.0             1
4                     0.0                   0.0             1
..                    ...                   ...           ...
423                   1.7                   2.2             1
424                   2.1                   2.2             1
425                   2.6                   1.8             1
426                   2.5                   1.9             1
427                   1.5                   1.8             1

[393 rows x 3 columns]

In [90]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.236170  0.670886   0.704481  0.670886  0.611060
1      AdaBoostClassifier:   0.134287  0.645570   0.776149  0.645570  0.539644
2    Voting_LR3_SVC1_ETC2:   0.421060  0.645570   0.668475  0.645570  0.569087
3  DecisionTreeClassifier:   0.004412  0.607595   0.584665  0.607595  0.578942
4  RandomForestClassifier:   0.088120  0.620253   0.598523  0.620253  0.581335
5    ExtraTreesClassifier:   0.130634  0.607595   0.579455  0.607595  0.562991
6      LogisticRegression:   0.006631  0.607595   0.369172  0.607595  0.459284
7                     svm:   0.012481  0.645570   0.776149  0.645570  0.539644
8         Stacking_scikit:   2.065085  0.670886   0.786521  0.670886  0.587109
9                  OvR_RF:   0.143811  0.620253   0.598523  0.620253  0.581335

# Teste 6 - 'Pré LDH (U/L)',  'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L'

In [91]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
                'Pré Albumina (g/dL)',
                'Pós Albumina (g/dL)', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)', 'Pós TFG   (ml/min/1.73 m2)',
                'Pré Ureia mg/dL', 'Pós Ureia mg/dL', 'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
               'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
                'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
                'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
                'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L']

col_df = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

Pré LDH (U/L)  Pós LDH (U/L)  Pré AST (U/L)  Pós AST (U/L)  \
0            629.0          796.0           33.0           48.0   
1            720.0         1182.0           49.0          105.0   
2            895.0         1531.0           57.0          145.0   
3            349.0          666.0           46.0          138.0   
4            643.0         1089.0           32.0           94.0   
..             ...            ...            ...            ...   
339          191.0          642.0           36.0          116.0   
340          190.0          751.0           29.0           59.0   
341          228.0          461.0           30.0           57.0   
342          246.0          545.0           31.0           82.0   
343          265.0          881.0           62.0          156.0   

     Pré Albumina (g/dL)  Pós Albumina (g/dL) Pré TGP (g/dL)  Pós TGP (g/dL)  \
0                    4.6                  4.0             37            63.0   
1                    4.9                  4.6             57            93.0   
2                    4.9                  4.8             58           113.0   
3                    4.8                  4.1             42           100.0   
4                    4.8                  4.3             32            85.0   
..                   ...                  ...            ...             ...   
339                  4.4                  4.7             38            42.0   
340                  4.4                  5.1             25            28.0   
341                  4.4                  5.1             29            43.0   
342                  4.5                  5.5             28            33.0   
343                  4.1                  5.2             44            51.0   

    Pré GGT U/L  Pós GGT U/L  ...  Pós Potássio mmol/L  Glicose pré  \
0            16         17.0  ...                  4.6          103   
1            38         37.0  ...                  4.3           95   
2            26         23.0  ...                  4.5           99   
3            13         10.0  ...                  4.6           94   
4            39         40.0  ...                  4.5           96   
..          ...          ...  ...                  ...          ...   
339          17         19.0  ...                  4.3           85   
340          19         22.0  ...                  4.7           91   
341          25         30.0  ...                  4.4           88   
342          17         18.0  ...                  5.0           93   
343          18         21.0  ...                  5.3          102   

     Glicose pós  Pré Sódio mmol/L  Pós Sódio mmol/L  Pré Magnésio mg/dL  \
0          110.0             145.0             146.0                 2.0   
1          114.0             142.0             141.0                 2.2   
2           98.0             145.0             146.0                 2.2   
3          144.0             144.0             143.0                 2.1   
4          142.0             142.0             142.0                 2.0   
..           ...               ...               ...                 ...   
339         91.0             139.0             137.0                 2.0   
340         94.0             139.0             138.0                 2.0   
341        101.0             140.0             138.0                 2.1   
342        107.0             138.0             139.0                 2.1   
343        102.0             139.0             140.0                 2.0   

     Pós Magnésio mg/dL  Pré Cloro mmol/L  Pós Cloro mmol/L  Rabdomiólise  
0                   2.0             103.0             106.0             0  
1                   1.9             102.0             103.0             1  
2                   2.2             105.0             100.0             1  
3                   2.0             106.0             108.0             1  
4                   1.9             102.0             103.0             1  
..               

In [92]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.332056  0.492754   0.495351  0.492754  0.453656
1      AdaBoostClassifier:   0.338263  0.840580   0.855603  0.840580  0.839163
2    Voting_LR3_SVC1_ETC2:   1.231620  0.782609   0.785032  0.782609  0.782335
3  DecisionTreeClassifier:   0.015436  0.811594   0.818655  0.811594  0.810800
4  RandomForestClassifier:   0.160835  0.811594   0.818655  0.811594  0.810800
5    ExtraTreesClassifier:   0.284345  0.840580   0.848198  0.840580  0.839908
6      LogisticRegression:   0.344146  0.782609   0.782938  0.782609  0.782609
7                     svm:   0.029335  0.826087   0.826087  0.826087  0.826087
8         Stacking_scikit:   2.607532  0.797101   0.806847  0.797101  0.795816
9                  OvR_RF:   0.136753  0.811594   0.818655  0.811594  0.810800

# Teste 7 - Série vermelha -  'Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %'

In [93]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %']

col_df = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

Pré RCB 10^6/mm³  Pós RCB 10^6/mm³  Pré HGB g/dL  Pós HGB g/dL  \
0                4.38              4.37          13.5          13.3   
1                0.00              4.64           0.0          13.9   
2                0.00              4.77           0.0          14.5   
3                5.22              4.52          14.9          13.1   
4                4.37              4.06          13.3          12.5   
..                ...               ...           ...           ...   
339              4.58              4.48          14.4          16.2   
340              4.49              5.61          13.7          11.8   
341              4.61              4.70          14.0          12.5   
342              4.64              3.24          13.8          12.3   
343              4.81              5.41          14.4          13.2   

     Pré HCT %  Pós HCT %  Pré MCV fL  Pós MCV fL  Pré MCH pg  Pós MCH pg  \
0         40.9       39.4        93.4        90.2        30.8        30.4   
1          0.0       40.6         0.0        87.5         0.0        30.0   
2          0.0       42.7         0.0        89.5         0.0        30.4   
3         47.7       38.7        91.4        85.6        28.5        29.0   
4         41.9       37.5        95.9        92.4        30.4        30.8   
..         ...        ...         ...         ...         ...         ...   
339       44.2       48.4        96.4        95.4        31.4        32.0   
340       42.2       36.5        93.8        93.7        30.4        30.3   
341       42.7       37.9        92.5        92.0        30.3        30.3   
342       41.7       37.3        89.8        89.6        29.8        29.5   
343       44.5       41.2        92.5        92.8        29.9        29.7   

     Pré MCHC g/dL  Pós MCHC g/dL  Pré PLT 10³/mm³  Pós PLT 10³/mm³  \
0             33.0           33.8            299.0            289.0   
1              0.0           34.2            268.0            304.0   
2              0.0           34.0              0.0            268.0   
3             31.2           33.9            242.0            246.0   
4             31.7           33.3            192.0            259.0   
..             ...            ...              ...              ...   
339            0.0            0.0              0.0              0.0   
340            0.0            0.0              0.0              0.0   
341            0.0            0.0              0.0              0.0   
342            0.0            0.0              0.0              0.0   
343            0.0            0.0              0.0              0.0   

     Pré RDW %  Pós RDW %  Pré RET %  Pós RET %  Rabdomiólise  
0         12.8       12.2       1.83       1.52             0  
1          0.0       13.4       0.00       1.47             1  
2          0.0       12.1       0.00       1.20             1  
3         12.9       12.4       0.00       1.11             1  
4         11.7       12.1       0.00       2.27             1  
..         ...        ...        ...        ...           ...  
339        0.0        0.0       0.00       1.56             1  
340        0.0        0.0       0.00       1.10             0  
341        0.0        0.0       0.00       0.96             0  
342        0.0        0.0       0.00       0.57             1  
343        0.0        0.0       0.00       1.67             1  

[344 rows x 19 columns]

In [94]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.286287  0.521739   0.524247  0.521739  0.488756
1      AdaBoostClassifier:   0.158838  0.681159   0.696457  0.681159  0.673601
2    Voting_LR3_SVC1_ETC2:   0.489763  0.666667   0.703509  0.666667  0.648909
3  DecisionTreeClassifier:   0.008610  0.594203   0.594178  0.594203  0.593691
4  RandomForestClassifier:   0.108350  0.695652   0.699213  0.695652  0.693720
5    ExtraTreesClassifier:   0.149289  0.681159   0.704538  0.681159  0.670328
6      LogisticRegression:   0.023774  0.579710   0.595271  0.579710  0.557320
7                     svm:   0.012672  0.695652   0.756689  0.695652  0.674663
8         Stacking_scikit:   2.086553  0.710145   0.766221  0.710145  0.692515
9                  OvR_RF:   0.135027  0.695652   0.699213  0.695652  0.693720

# Teste 8 - Série leucocitária - 'Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %'

In [95]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %']

col_df = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

Pré WBC 10³/mm³  Pós WBC 10³/mm³  Pré NEUT %  Pós NEUT %  Pré LINF %  \
0               7.38             8.17        49.1        72.6        39.0   
1               0.00             9.38         0.0        72.0         0.0   
2               0.00             7.48         0.0        82.8         0.0   
3               4.91             5.79        52.0        67.8        34.6   
4               6.31            10.56        79.1        82.6        15.2   
..               ...              ...         ...         ...         ...   
442             6.63             8.28        59.1        55.6        30.3   
443             8.61             8.28        59.5        70.7        31.5   
444             5.45            15.77        44.6        52.3        41.8   
445             4.90            11.69        57.4        57.5        31.0   
446             7.57             9.16        19.1        42.4        69.6   

     Pós LINF %  Pré MONO %  Pós MONO %  Pré EOS %  Pós EOS %  Pré BASO %  \
0          21.1         8.3         5.3        3.3        0.9         0.3   
1          12.5         0.0        15.0        0.0        0.3         0.0   
2          11.2         0.0         5.6        0.0        0.1         0.0   
3          21.6        11.2         6.2        1.2        3.5         1.0   
4          10.9         4.1         5.5        1.1        0.9         0.5   
..          ...         ...         ...        ...        ...         ...   
442        34.1         8.0         8.5        1.4        1.0         1.2   
443        20.6         6.7         8.3        2.0        0.1         0.3   
444        36.3         9.0        10.4        3.9        0.6         0.7   
445        31.4         8.8         9.9        2.0        0.7         0.8   
446        48.8         8.7         7.8        1.7        0.5         0.9   

     Pós BASO %  Rabdomiólise  
0           0.1             0  
1           0.2             1  
2           0.3             1  
3           0.9             1  
4           0.1             1  
..          ...           ...  
442         0.8             0  
443         0.3             0  
444         0.4             0  
445         0.5             0  
446         0.5             0  

[411 rows x 13 columns]

In [96]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.301419  0.686747   0.685893  0.686747  0.685919
1      AdaBoostClassifier:   0.151954  0.686747   0.685908  0.686747  0.684710
2    Voting_LR3_SVC1_ETC2:   0.507754  0.686747   0.701756  0.686747  0.672464
3  DecisionTreeClassifier:   0.007877  0.662651   0.661898  0.662651  0.658735
4  RandomForestClassifier:   0.118004  0.686747   0.685908  0.686747  0.684710
5    ExtraTreesClassifier:   0.152625  0.722892   0.723330  0.722892  0.720427
6      LogisticRegression:   0.022020  0.710843   0.710438  0.710843  0.708964
7                     svm:   0.015922  0.590361   0.632415  0.590361  0.519485
8         Stacking_scikit:   1.900387  0.722892   0.733673  0.722892  0.714534
9                  OvR_RF:   0.145148  0.686747   0.685908  0.686747  0.684710